In [12]:
from typing import List, Optional
import sky
import os


In [13]:
# Total Nodes, INCLUDING Head Node
num_nodes = 2

In [17]:
workdir =  os.getcwd()
print(workdir)

/home/jupyter/skypilot-gke/ray-train


In [18]:
# The setup command.  Will be run under the working directory.
setup = '''
  pip install "ray[train]"
  pip install tqdm
  pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
  sudo mkdir ./checkpoint
  '''


In [19]:
# The command to run.  Will be run under the working directory.
def run_fn(node_rank: int, ip_list: List[str]) -> Optional[str]:
    num_nodes = len(ip_list)
    return f"""\
    if [ "{node_rank}" == "0" ]; then
      ps aux | grep ray | grep 6379 &> /dev/null || ray start --head  --disable-usage-stats --port 6379
      sleep 5
      python train.py --num-workers {num_nodes}
    else
      sleep 5
      ps aux | grep ray | grep 6379 &> /dev/null || ray start --address {ip_list[0]}:6379 --disable-usage-stats
    fi
    """

In [20]:
train = sky.Task(
    'train',
    setup=setup,
    workdir=workdir,
    num_nodes=num_nodes,
    run=run_fn,
)

In [21]:
train.set_resources({
    ##### Fully specified
    sky.Resources(sky.Kubernetes(), accelerators={'T4': 1})
})

Task<name=train>(run=<fn>)
  nodes: 2
  resources: Kubernetes({'T4': 1})

In [22]:
sky.launch(train, cluster_name='ray-train')

⠴ Syncing

I 05-27 21:42:00 cloud_vm_ray_backend.py:3217] Running setup on 2 nodes.


Looking in indexes: https://download.pytorch.org/whl/cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.6/839.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.6/839.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 117.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 117.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 62.0 MB/s eta 0:00:0

I 05-27 14:44:21 log_lib.py:408] Start streaming logs for job 1.
INFO: Tip: use Ctrl-C to exit log streaming (task will not be killed).
INFO: Waiting for task resources on 2 nodes. This will block if the cluster is full.
INFO: All task resources reserved.
INFO: Reserved IPs: ['10.12.1.5', '10.12.2.5']
(worker1, rank=1, pid=833, ip=10.12.1.5) 2024-05-27 14:44:24,535	INFO usage_lib.py:423 -- Usage stats collection is disabled.
(worker1, rank=1, pid=833, ip=10.12.1.5) 2024-05-27 14:44:24,535	INFO scripts.py:744 -- Local node IP: 10.12.1.5
(worker1, rank=1, pid=833, ip=10.12.1.5) 2024-05-27 14:44:27,884	SUCC scripts.py:781 -- --------------------
(worker1, rank=1, pid=833, ip=10.12.1.5) 2024-05-27 14:44:27,884	SUCC scripts.py:782 -- Ray runtime started.
(worker1, rank=1, pid=833, ip=10.12.1.5) 2024-05-27 14:44:27,884	SUCC scripts.py:783 -- --------------------
(worker1, rank=1, pid=833, ip=10.12.1.5) 2024-05-27 14:44:27,884	INFO scripts.py:785 -- Next steps
(worker1, rank=1, pid=833, ip=10

Connection to 10.12.2.5 closed.


Clusters
NAME       LAUNCHED    RESOURCES                                 STATUS  AUTOSTOP  COMMAND                       
ray-train  6 mins ago  2x Kubernetes(2CPU--8GB--1T4, {'T4': 1})  UP      -         /opt/conda/lib/python3.10...  



(1,
 ResourceHandle(
 	cluster_name=ray-train,
 	cluster_name_on_cloud=ray-train-a861,
 	head_ip=10.12.2.5,
 	stable_internal_external_ips=[('10.12.2.5', '10.12.2.5'), ('10.12.1.5', '10.12.1.5')],
 	stable_ssh_ports=[22, 22],
 	cluster_yaml=/root/.sky/generated/ray-train.yml, 
 	launched_resources=2x Kubernetes(2CPU--8GB--1T4, {'T4': 1}), 
 	docker_user=None,
 	ssh_user=sky,
 	tpu_create_script=None, 
 	tpu_delete_script=None))